# JSON 확인 과정

In [1]:
#-*- coding: EUC-KR -*-

%matplotlib inline 

import numpy as np 
import pandas as pd
import skimage.io as io 
import matplotlib.pyplot as plt 
import pylab 
import json
from tqdm import tqdm
import glob
import os
import cv2

pylab.rcParams['figure.figsize'] = (8.0, 10.0)

rootpath = "D:/F1soft 120해커톤"

In [2]:
# row 생략 없이 출력
pd.set_option('display.max_rows', None)
# col 생략 없이 출력
pd.set_option('display.max_columns', None)

In [3]:
train_json_list = [] 

for (path, dir, files) in os.walk(rootpath):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.json':
            full=path+'/'+filename
            train_json_list.append(full)

In [4]:
train_mp4_list = [] 

for (path, dir, files) in os.walk(rootpath):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.mp4':
            full=path+'/'+filename
            train_mp4_list.append(full)

In [5]:
json_data_list=[]
mp4_fps_list=[]
mp4_framecnt_list=[]

In [6]:
for anno in tqdm(train_json_list): # annotation.json 파일 읽어들이기
    with open(anno, encoding='utf-8') as json_file:
        json_data_list.append(json.load(json_file))


100%|████████████████████████████████████████████████████████████████████████████████| 355/355 [00:38<00:00,  9.24it/s]


In [7]:
for vid_info in tqdm(train_mp4_list): # annotation.mp4 파일 읽어들이기
    cap = cv2.VideoCapture(vid_info) 
    codec = cv2.VideoWriter_fourcc(*'XVID')
    vid_size = (round(cap.get(cv2.CAP_PROP_FRAME_WIDTH)),round(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    vid_fps = cap.get(cv2.CAP_PROP_FPS )
    frame_cnt = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))    
    mp4_fps_list.append(vid_fps)
    mp4_framecnt_list.append(frame_cnt) 

100%|████████████████████████████████████████████████████████████████████████████████| 355/355 [00:23<00:00, 14.79it/s]


# 원본 영상과 json 파일 프레임 일치 확인

In [8]:
frame_check_df=pd.DataFrame(columns=['vid','annotation'])

frame_check_df=pd.DataFrame()

for i in range(len(mp4_framecnt_list)):
    made_row=pd.DataFrame([mp4_framecnt_list[i],len(json_data_list[i]['annotations']['anno_info'])]).T
    frame_check_df=pd.concat([frame_check_df,made_row],axis=0)
    
frame_check_df.reset_index(drop=True,inplace=True)
frame_check_df.rename(columns={0:'vid',1:'annotation'},inplace= True)

In [9]:
frame_check_df['nomatch'] = frame_check_df.apply(lambda x: x['annotation']-x['vid'],axis=1)

In [10]:
frame_check_df['json'] = train_json_list

In [11]:
frame_check_df['mp4'] = train_mp4_list

In [15]:
good_fps=frame_check_df[(frame_check_df['nomatch']<=10) & (frame_check_df['nomatch']>=0)] # 0,5,6

In [19]:
bad_fps=frame_check_df[(frame_check_df['nomatch']>10) | (frame_check_df['nomatch']<0)] # 0,5,6

In [24]:
bad_list=[]
bad_list=bad_fps['mp4'].to_list()

In [27]:
sort_bad_list=[]
for i in bad_list:
    sort_bad_list.append(i.split('/')[-1])

In [132]:
train_json_list=good_fps['json'].to_list()
train_mp4_list=good_fps['mp4'].to_list()

In [187]:
train_json_list.remove('D:/F1soft 120해커톤\\2 동승자 탑승 위반\\2 킥보드\\1 위반/T002628_000_1_2_2.json')
train_mp4_list.remove('D:/F1soft 120해커톤\\2 동승자 탑승 위반\\2 킥보드\\1 위반/T002628_000_1_2_2.mp4')

In [134]:
for anno in tqdm(train_json_list): # annotation.json 파일 읽어들이기
    with open(anno, encoding='utf-8') as json_file:
        json_data_list.append(json.load(json_file))


100%|████████████████████████████████████████████████████████████████████████████████| 255/255 [00:23<00:00, 10.68it/s]


# 객체 빈도 수 데이터프레임 생성 

In [135]:
def getTypeName(dic):
    '''
    Parameters:
    -----------
    dic: result of readJSON
    
    Returns:
    --------
    pd.DataFrame
        dataframe with image_id as index and number of each objects contained in the image as values
    '''
    
    output = {}
    image_id = dic['info']['video_id']+'_'+dic['info']['clip_id']
    
    for i in range(len(dic['annotations']['anno_info'])):
        for j in range(len(dic['annotations']['anno_info'][i]['frame_annotation'])):
            type_name = dic['annotations']['anno_info'][i]['frame_annotation'][j]['category_name']

            if output.get(type_name) == None :
                output[type_name] = 1

            else :
                output[type_name] += 1
    
    for i in dic['description']:
        type_name=i
        output[i]=dic['description'][i]
    
    
    return pd.DataFrame(output, index = [image_id])

In [136]:
%%time

total_df = pd.DataFrame()

for path in tqdm(json_data_list):
    
    total_df = pd.concat([total_df, getTypeName(path)], axis = 0, join = 'outer')
    
total_df.fillna(0, inplace = True)

100%|███████████████████████████████████████████████████████████████████████████████| 255/255 [00:01<00:00, 160.71it/s]

Wall time: 1.59 s


In [54]:
cate_name=total_df.columns

In [60]:
list_category=[]

for i,j in zip(cate_id,cate_name):
    if i not in ['violation','PM','violation_type','duration']:
        list_category.append([i,j])

In [62]:
sdict = sorted(list_category,key = lambda x : int(x[0]))


[['3', '02_자전거 겸용_01_인도_01_도로유형'],
 ['2', '01_일반_01_인도_01_도로유형'],
 ['6', '05_일반 차로_02_차도_01_도로유형'],
 ['18', '17_직진_05_노면표시_02_노면유형'],
 ['17', '16_자전거 전용도로_05_노면표시_02_노면유형'],
 ['132', '61_자전거표지판_위반_06_표지판_03_도로 시설물'],
 ['56', '55_자전거탑승자_09_Rider_04_이동체'],
 ['55', '54_오토바이탑승자_09_Rider_04_이동체'],
 ['22', '21_직진 및 우회전_05_노면표시_02_노면유형'],
 ['19', '18_좌회전_05_노면표시_02_노면유형'],
 ['8', '07_안전지대_03_차선_02_노면유형'],
 ['7', '06_중앙차선_03_차선_02_노면유형'],
 ['33', '32_직진 및 좌회전_06_표지판_03_도로 시설물'],
 ['4', '03_횡단보도_01_인도_01_도로유형'],
 ['9', '08_정지선_04_정지선_02_노면유형'],
 ['126', '56_킥보드탑승자_위반_09_Rider_04_이동체'],
 ['5', '04_자전거 전용도로_02_차도_01_도로유형'],
 ['54', '53_킥보드_08_PM_04_이동체'],
 ['20', '19_우회전_05_노면표시_02_노면유형'],
 ['46', '45_보행자 신호등(적색)_07_신호등_03_도로 시설물'],
 ['43', '42_비보호좌회전_06_표지판_03_도로 시설물'],
 ['47', '46_차량 신호등 원형등화(녹색)_07_신호등_03_도로 시설물'],
 ['53', '52_자전거_08_PM_04_이동체'],
 ['131', '61_자전거표지판_06_표지판_03_도로 시설물'],
 ['21', '20_직진 및 좌회전_05_노면표시_02_노면유형'],
 ['125', '55_자전거탑승자_위반_09_Rider_04_이동체'],
 ['49', '48_차량 신호등 원형등화(적색)

In [1]:
ca_id_list=[]

In [36]:
for i,num in zip(sdict,range(len(sdict))):
    ca_id_list.append(int(i[0]))

In [57]:
import pickle

with open ("D:/category_id_name", "wb") as file:
    pickle.dump(sdict, file)

In [148]:
train_json_list = [] 

for (path, dir, files) in os.walk(rootpath):
    for filename in files:
        ext = os.path.splitext(filename)[-1]
        if ext == '.json':
            full=path+'/'+filename
            train_json_list.append(full)